In [ ]:
print("hello")

In [ ]:
import os
import requests
from datetime import datetime, timedelta, timezone
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

In [ ]:
exp_id = 35
warmup_sec = 60
threshold = 2000 #slope


In [ ]:
#exp_id = 35

#os.chdir("./results-new")

raw_runs = []

filenames = [filename for filename in os.listdir('.') if filename.startswith(f"exp{exp_id}") and filename.endswith(".csv")]
for filename in filenames:
    #print(filename)
    run_params = filename[:-4].split("_")
    dim_value = run_params[2]
    instances = run_params[3]

    df = pd.read_csv(filename)
    input = df.loc[df['topic'] == "input"]
    #print(input)
    input['sec_start'] = input.loc[0:, 'timestamp'] - input.iloc[0]['timestamp']
    #print(input)
    #print(input.iloc[0, 'timestamp'])
    regress = input.loc[input['sec_start'] >= warmup_sec] # Warm-Up
    #regress = input

    #input.plot(kind='line',x='timestamp',y='value',color='red')
    #plt.show()

    X = regress.iloc[:, 2].values.reshape(-1, 1)  # values converts it into a numpy array
    Y = regress.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions

    trend_slope = linear_regressor.coef_[0][0]
    #print(linear_regressor.coef_)

    row = {'dim_value': int(dim_value), 'instances': int(instances), 'trend_slope': trend_slope}
    #print(row)
    raw_runs.append(row)

runs = pd.DataFrame(raw_runs)


In [ ]:
runs.head()

In [ ]:
#threshold = 1000

# Set to true if the trend line has a slope less than 
runs["suitable"] =  runs.apply(lambda row: row['trend_slope'] < threshold, axis=1)

runs.columns = runs.columns.str.strip()
runs.sort_values(by=["dim_value", "instances"])

In [ ]:
filtered = runs[runs.apply(lambda x: x['suitable'], axis=1)]

grouped = filtered.groupby(['dim_value'])['instances'].min()
min_suitable_instances = grouped.to_frame().reset_index()

min_suitable_instances

In [ ]:
min_suitable_instances.plot(kind='line',x='dim_value',y='instances')

plt.show()